In [ ]:
execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
from analysis.lib.m2.ssro import ssro, sequence
import scipy.fftpack
import scipy.signal
import win32com.client #imports the pywin32 library

%matplotlib inline

In [ ]:
def fourier(signal, time):
    """Returns the Fouier transform of a single signal with time."""
    t_step = (time[-1] - time[0])/time.size
    fft = scipy.fftpack.fft(signal) * t_step
    freq = scipy.fftpack.fftfreq(time.size, t_step)
    n = len(fft)
    fft = fft[0:n/2]
    freq = freq[0:n/2]
    return np.abs(fft), freq

def read_data():
    # Get the latest data from the folder.
    fol = tb.latest_data('mw_pulse_msmt', folder='Z:/data')
#     fol = tb.data_from_time("20180319144419", folder='Z:/data')
    f = sequence.SequenceAnalysis(fol, hdf5_mode='r') 
    waveform = np.array(f.g['oscilloscope_msmt']['data'])
    global number_of_pulses
    number_of_pulses = int(np.array(f.g['oscilloscope_msmt']['number_of_pulses']))
    points_per_pulse = int(1e6 // number_of_pulses)
    # Make numpy arrays to store the information in and make them global.
    global t, amplitude, fft, freq
    t = np.zeros((number_of_pulses, points_per_pulse))
    amplitude = np.zeros((number_of_pulses, points_per_pulse))
    fft = np.zeros((number_of_pulses, points_per_pulse//2))
    freq = np.zeros((number_of_pulses, points_per_pulse//2))
    # Get the information of all the pulses
    for i in range(number_of_pulses):
        t[i] = waveform[0][i*points_per_pulse:(i+1)*points_per_pulse]
        amplitude[i] = waveform[1][i*points_per_pulse:(i+1)*points_per_pulse]    
        fft[i], freq[i] = fourier(amplitude[i], t[i])

def hermite_pulse(env_amplitude=1, mu=0, T_herm = 1.667e-8, freq_sim = 1.719219e9):
    t = np.linspace(-25e-8,25e-8,int(1e4))
    env = env_amplitude*(1-0.667*((t-mu)/T_herm)**2)*np.exp(-((t-mu)/T_herm)**2)
    pulse = env * np.sin(t*2*np.pi*freq_sim)
    fft, freq = fourier(pulse, t)
    return pulse, t, fft, freq
        
def plot_ideal_fft(amplitude=1):
    pulse, t, fft, freq = hermite_pulse(amplitude)
    plt.plot(freq, fft, 'r--', label='Simulation')
    
def set_limits():
    ind = np.argmin(np.abs(freq[0] - 1.72e9))
    plt.xlim(1.66e9, 1.8e9)
    plt.ylim(0, 1.1*fft[0, ind])

# Noise of oscilloscope

In [ ]:
# read_data()
# noise_oscilloscope = np.std(amplitude)

# Analysis of one (first) pulse

In [ ]:
read_data()
t = t[0]
amplitude = amplitude[0]
# Plot waveform
plt.plot(t, amplitude)
plt.xlabel('s')
plt.ylabel('V')
plt.title('Single Hermite pulse')
middle = (t[-1]-t[0])/2 + t[0]
plt.xlim(middle-1e-7, middle+1e-7)
plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
plt.show()

# Plot the fourier transform
plt.plot(freq[0], fft[0], label='Data')
plot_ideal_fft(2.22e-1)
plt.title('Fourier transform')
plt.xlabel('Frequency [Hz]')
plt.legend()
set_limits()
plt.show()

# Analysis of the pulses in one sweep

In [ ]:
read_data()
for i in range(number_of_pulses):
    plt.plot(freq[i], fft[i])
set_limits()
plt.show()

In [ ]:
read_data()
mean = np.mean(fft, axis=0)
std = np.std(fft, axis=0)
plt.figure(figsize=(10,8))
plt.plot(freq[0], mean, 'k')
plt.fill_between(freq[0],  mean - std, mean + std, facecolor='blue', alpha=0.5)
set_limits()
plt.show()

plt.figure(figsize=(10,3))
plt.plot(freq[0],  std, label="Noise on RF signal")
# plt.plot([1.66e-9, 1.8e-9], [noise_oscilloscope, noise_oscilloscope], 'k--', label="Noise oscilloscope")
plt.xlim(1.66e9, 1.8e9)
plt.show()
ind_low = int(np.argmin(np.abs(freq[0] - 1.705e9)))
ind_high = int(np.argmin(np.abs(freq[0] - 1.735e9)))
mean_plateau = np.mean(mean[ind_low:ind_high])
print "The relative mean standard deviation to the peak is {:.3e}.".format(np.mean(std)/mean_plateau)